In [7]:
import comet_ml
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
import datasets
from transformers import logging, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from itertools import combinations
import numpy as np
import api_keys
import re

In [8]:
experiment = comet_ml.Experiment(api_key=api_keys.comet, project_name='rap-lyrics-generator-llm')

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : poised_hamburger_607
COMET INFO:     url                   : https://www.comet.com/artaasd95/rap-lyrics-generator-llm/13404e1c0d7241b4baf92d2c6d045d77
COMET INFO:   Parameters:
COMET INFO:     num_train_epochs            : 3
COMET INFO:     per_device_train_batch_size : 2
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (519 bytes)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: 
COMET WARNING: As you 

COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


In [9]:
experiment.log_parameters({
    "num_train_epochs": 5,
    "per_device_train_batch_size": 2,
    # Add any other relevant hyperparameters here
})

In [11]:
rap_lyrics_train_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')
#rap_lyrics_train_dataset = rap_lyrics_train_dataset[:int(len(rap_lyrics_train_dataset)*0.7)]
#rap_lyrics_test_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')[int(len(rap_lyrics_train_dataset)*0.7):]

In [15]:
model_name = "gpt2"  # You could use a larger model like gpt2-medium for better performance
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

In [16]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=8092)

# Tokenize dataset
tokenized_train_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create a data collator for dynamic batching
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# tokenized_test_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# data_collator_test = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/7319 [00:00<?, ? examples/s]

In [17]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="comet_ml",  # Ensure training logs are sent to Comet
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    #eval_dataset=tokenized_test_dataset,
    compute_metrics=None,  # Add compute_metrics function if needed
)

# Fine-tune the model
trainer.train()

COMET INFO: An experiment with the same configuration options is already running and will be reused.


  0%|          | 0/18300 [00:00<?, ?it/s]

{'loss': 3.2603, 'grad_norm': 15.812793731689453, 'learning_rate': 4.9726775956284156e-05, 'epoch': 0.03}
{'loss': 2.988, 'grad_norm': 11.727437019348145, 'learning_rate': 4.945355191256831e-05, 'epoch': 0.05}
{'loss': 3.1803, 'grad_norm': 7.573428630828857, 'learning_rate': 4.918032786885246e-05, 'epoch': 0.08}
{'loss': 3.1779, 'grad_norm': 8.897269248962402, 'learning_rate': 4.890710382513661e-05, 'epoch': 0.11}
{'loss': 3.1598, 'grad_norm': 10.217747688293457, 'learning_rate': 4.863387978142076e-05, 'epoch': 0.14}
{'loss': 3.0267, 'grad_norm': 5.547766208648682, 'learning_rate': 4.836065573770492e-05, 'epoch': 0.16}
{'loss': 2.9938, 'grad_norm': 4.865967273712158, 'learning_rate': 4.808743169398907e-05, 'epoch': 0.19}
{'loss': 3.1659, 'grad_norm': 7.977914333343506, 'learning_rate': 4.7814207650273224e-05, 'epoch': 0.22}
{'loss': 2.9804, 'grad_norm': 5.74665641784668, 'learning_rate': 4.754098360655738e-05, 'epoch': 0.25}
{'loss': 2.999, 'grad_norm': 5.771641254425049, 'learning_rat

TrainOutput(global_step=18300, training_loss=2.4482092985559682, metrics={'train_runtime': 2347.904, 'train_samples_per_second': 15.586, 'train_steps_per_second': 7.794, 'total_flos': 4780638823680000.0, 'train_loss': 2.4482092985559682, 'epoch': 5.0})

In [18]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_rap_model')
tokenizer.save_pretrained('./fine_tuned_rap_model')

('./fine_tuned_rap_model\\tokenizer_config.json',
 './fine_tuned_rap_model\\special_tokens_map.json',
 './fine_tuned_rap_model\\vocab.json',
 './fine_tuned_rap_model\\merges.txt',
 './fine_tuned_rap_model\\added_tokens.json',
 './fine_tuned_rap_model\\tokenizer.json')

In [19]:
# metrics = trainer.evaluate(eval_dataset=tokenized_test_dataset)
# experiment.log_metrics(metrics)

In [20]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : practical_hearth_893
COMET INFO:     url                   : https://www.comet.com/artaasd95/rap-lyrics-generator-llm/cc4cab51aa9944e1a4e6f8dcd45a3d30
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [1830]                    : (0.29766061902046204, 4.7499237060546875)
COMET INFO:     train/epoch [184]              : (0.0273224043715847, 5.0)
COMET INFO:     train/grad_norm [183]          : (2.5700201988220215, 15.812793731689453)
COMET INFO:     train/learning_rate [183]      : (0.0, 4.9726775956284156e-05)
COMET INFO:     train/loss [183]               : (2.0152, 3.2603)
COMET INFO:     train/total_flos               : 478063882368000